# ST SPV1050 Part Selection

## MPPT Setting 
Will set R1, R2, and R3 with the MPPT from the [solar cell](https://waf-e.dubudisk.com/anysolar.dubuplus.com/techsupport@anysolar.biz/O18Ae0B/DubuDisk/www/Gen3/SM141K04LV%20DATA%20SHEET%20202007.pdf)

### Specs 
* VMpp 2.23V
* Voc 2.76V
* Vuvp (undervoltage protection threshold) 3.5V
* Vmpp ratio = 0.85
* Imp -= 55mA

### Equations

$$ V_{uvp} \geq V_{oc}\frac{R2+R3}{R1+R2+R3} $$ 

$$ V_{mpp ratio} = \frac{R3}{R1+R2+R3} $$

$$ 10\,M\Omega \leq R1+R2+R3 \leq 20\,M\Omega $$


### Solution
Since our solar cell will operate above the battery voltage we can set $R1 = 0\Omega$ and can solve the equations for R3 and then R2
Then we get 
* R3 = 8.5
* R2 = 1.5



In [18]:
vup = 3.5
voc = 6.91
vmpp = 5.58
r1  = 10
vup_min = 2.2
veoc = 4.2

#e5 in data sheet
e5_sum = 15 * (vup_min/voc)
e6_sum = 15 * 51 * vmpp / veoc

print(e5_sum)
print(e6_sum)

mpp_ratio = vmpp/voc

r3 = mpp_ratio * (min([e5_sum, e6_sum])) 

r2 = min([e5_sum, e6_sum]) - r3

print(r2)
print(r3)

# these are e96 values closest to the calculated ones
r3a = 3.83
r2a = .909

mpp_ratio_a = r3a/(r3a + r2a)

print(mpp_ratio)
print(mpp_ratio_a)

(r3a+r2a) < ((r1+r2a+r3a)*vup_min/voc)

4.7756874095513755
1016.3571428571428
0.9191988791177033
3.8564885304336722
0.8075253256150506
0.8081873813040726
14.739


## Input Capacitance

Ever 16 seconds the SVP1050 samples the open circuit voltage for 400ms. In this time, the input cap C1 is charged to VOC. This can be modeled with an equvilant resistance 


$$ R_{eq} = \frac{V_{oc}-V_{mpp}}{I_{mpp}} $$ 

and thus 

$$ C_1 \leq \frac{400\,{ms}}{R_{eq}} $$

So if C1 is too small we will over charge C1 for the current provided but if it is too small then we will undercharge and be of the mpp 

In [28]:
import matplotlib as plt

imp = 0.0001
t1 = 0.4

req = (voc - vmpp)/imp 

c1_max = t1/req
print(c1_max)

7.547169811320758e-05


## Under Voltage and End of Charge Setting

Under voltage is the battery cuttoff voltage data sheet for battery has it as 3.0V we will set it to 3.1V to be safe

End of Charge is the maxium charge voltage for the battery and itll be set to 4.2V according to the battery data sheet

Vbg is the band-gap reference voltage and it is used to set eh uvp and eco and is 1.23V

$$ V_{bg} = V_{uvp}\frac{R5+R6}{R4+R5+R6} $$
$$ V_{bg} = V_{eoc}\frac{R6}{R4+R5+R6} $$

For low loss 
$$ 10\,M\Omega \leq R4+R5+R6 \leq 20\,M\Omega $$


* r4 = 6M
* r5 = 1M
* r6 = 3M

In [29]:
v_uvp_ratio = 1.23/3.1

v_eoc_ratio = 1.23/4.2

In [32]:
R5_rat  = v_uvp_ratio - v_eoc_ratio

In [33]:
R5_rat

0.10391705069124424

In [40]:
rt= 10
r6 = v_eoc_ratio*rt
r5 = R5_rat*rt
r4 = rt-r6-r5

print(r4)
print(r5)
print(r6)

6.03225806451613
1.0391705069124424
2.928571428571428


In [43]:
# tget actual values with rounding 
r6a = 3
r5a = 1
r4a = 6

vuvpa = 1.23/((r5a+r6a)/rt)
veoca = 1.23/(r6a/rt)
print(vuvpa)
print(veoca)

3.0749999999999997
4.1000000000000005
